In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
194,AFG,Asia,Afghanistan,2020-07-22,35727.0,112.0,1190.0,4.0,917.763,2.877,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
330,ALB,Europe,Albania,2020-07-22,4290.0,119.0,117.0,4.0,1490.722,41.351,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
530,DZA,Africa,Algeria,2020-07-22,24278.0,587.0,1100.0,13.0,553.647,13.386,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-21,France,176940.0,0.0,30183.0,0.0,0.0,0.0
0,2020-07-21,China,85802.0,0.0,4654.0,0.0,0.0,0.0
0,2020-07-21,Italy,244964.0,0.0,35085.0,0.0,0.0,0.0
0,2020-07-21,Spain,264843.0,0.0,28434.0,0.0,0.0,0.0
0,2020-07-21,United States,3894733.0,0.0,141553.0,0.0,0.0,0.0
0,2020-07-21,World,14995909.0,0.0,616085.0,0.0,0.0,0.0
0,2020-07-21,United Kingdom,296362.0,0.0,45399.0,0.0,0.0,0.0
0,2020-07-21,Germany,202680.0,0.0,9101.0,0.0,0.0,0.0
0,2020-07-21,Iran,279595.0,0.0,14586.0,0.0,0.0,0.0
0,2020-07-21,Turkey,221461.0,0.0,5523.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-22,France,176940.0,177338.0,30183.0,30165.0,-398.0,18.0
0,2020-07-22,China,85802.0,85771.0,4654.0,4648.0,31.0,6.0
0,2020-07-22,Italy,244964.0,244752.0,35085.0,35073.0,212.0,12.0
0,2020-07-21,Spain,264843.0,266194.0,28434.0,28424.0,-1351.0,10.0
0,2020-07-22,United States,3894733.0,3902058.0,141553.0,142066.0,-7325.0,-513.0
0,2020-07-22,World,14995909.0,14890516.0,616085.0,616317.0,105393.0,-232.0
0,2020-07-22,United Kingdom,296362.0,295817.0,45399.0,45422.0,545.0,-23.0
0,2020-07-22,Germany,202680.0,202799.0,9101.0,9095.0,-119.0,6.0
0,2020-07-22,Iran,279595.0,278827.0,14586.0,14634.0,768.0,-48.0
0,2020-07-22,Turkey,221461.0,221500.0,5523.0,5526.0,-39.0,-3.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")